# A4: World Development Indicators (WDI) - Exploratory Visual Analysis
Look at a dataset we provide → come up with a question → iteratively answer the question with visualizations → write up your process, describing your question and corresponding visualizations 

## 0 Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import geopandas as gpd

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from pca import pca

import warnings
warnings.filterwarnings("ignore")

## 1 Data Preprocessing
### 1.1 Read datasets
#### 1.1.1 WDICSV

In [2]:
df_main = pd.read_csv("WDI_dataset/WDICSV.csv")
print(df_main.shape)
df_main.head(2)

(392882, 67)


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,16.914625,17.392349,17.892005,18.359993,18.795151,19.295176,19.788156,20.279599,20.773627,NaN
1,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,6.473301,6.720331,7.015917,7.281390,7.513673,7.809566,8.075889,8.366010,8.684137,NaN


#### 1.1.2 WDICountry

In [3]:
df_country = pd.read_csv("WDI_dataset/WDICountry.csv")
print(df_country.shape)
df_country.head(2)

(265, 31)


,Country Code,Short Name,Table Name,Long Name,2-alpha code,Currency Unit,Special Notes,Region,Income Group,WB-2 code,...,Government Accounting concept,IMF data dissemination standard,Latest population census,Latest household survey,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data
0,ABW,Aruba,Aruba,Aruba,AW,Aruban florin,NaN,Latin America & Caribbean,High income,AW,...,NaN,Enhanced General Data Dissemination System (e-...,2020 (expected),NaN,NaN,Yes,NaN,NaN,2018.0,NaN
1,AFE,Africa Eastern and Southern,Africa Eastern and Southern,Africa Eastern and Southern,ZH,NaN,"26 countries, stretching from the Red Sea in t...",NaN,NaN,ZH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 1.1.3 WDISeries

In [4]:
df_indicators = pd.read_csv("WDI_dataset/WDISeries.csv")
print(df_indicators.shape)
df_indicators.head(2)

(1477, 20)


,Series Code,Topic,Indicator Name,Short definition,Long definition,Unit of measure,Periodicity,Base Period,Other notes,Aggregation method,Limitations and exceptions,Notes from original source,General comments,Source,Statistical concept and methodology,Development relevance,Related source links,Other web links,Related indicators,License Type
0,AG.AGR.TRAC.NO,Environment: Agricultural production,"Agricultural machinery, tractors",NaN,Agricultural machinery refers to the number of...,NaN,Annual,NaN,NaN,Sum,The data are collected by the Food and Agricul...,NaN,NaN,"Food and Agriculture Organization, electronic ...",A tractor provides the power and traction to m...,Agricultural land covers more than one-third o...,NaN,NaN,NaN,CC BY-4.0
1,AG.CON.FERT.PT.ZS,Environment: Agricultural production,Fertilizer consumption (% of fertilizer produc...,NaN,Fertilizer consumption measures the quantity o...,NaN,Annual,NaN,The world and regional aggregate series do not...,Weighted average,The FAO has revised the time series for fertil...,NaN,NaN,"Food and Agriculture Organization, electronic ...",Fertilizer consumption measures the quantity o...,"Factors such as the green revolution, has led ...",NaN,NaN,NaN,CC BY-4.0


### 1.2 Clean datasets
#### 1.2.1 WDICSV

In [5]:
df_main.drop(["Indicator Code"], axis=1, inplace=True)
df_main = df_main.fillna('N/A') # Replace blank with N/A
print(df_main.columns) # Print the columns to check if we need to melt

Index(['Country Name', 'Country Code', 'Indicator Name', '1960', '1961',
       '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970',
       '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979',
       '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988',
       '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
       '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2018', '2019', '2020', '2021', '2022'],
      dtype='object')


In [6]:
# We need to melt
df_main_clean = df_main.melt(id_vars=["Country Name", "Country Code", "Indicator Name"], var_name='Year', value_name='Indicator_Value') # Year(column -> row)
df_main_clean.head(2)

,Country Name,Country Code,Indicator Name,Year,Indicator_Value
0,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,1960,N/A
1,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,1960,N/A


In [7]:
df_summary = df_main_clean.describe()
# df_summary.to_csv("WDI_test/WDICSV_summary.csv", index=False)

# Check if there are any columns with just Null values
blank_columns = df_summary.columns[df_summary.loc["count"] == 0]
if blank_columns > 0:
    print(blank_columns)

In [8]:
df_main_clean.columns.name = None
df_main_clean = df_main_clean.fillna('N/A') # Replace null values with N/A
df_main_clean.head(2)

,Country Name,Country Code,Indicator Name,Year,Indicator_Value
0,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,1960,N/A
1,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,1960,N/A


In [9]:
# df_main_clean.to_csv("WDI_test/WDICSV_clean.csv", index=False)

#### 1.2.2 WDICountry

In [10]:
df_country = df_country.fillna('N/A')
df_country_clean = df_country.drop(["Currency Unit", "Short Name", "Table Name", "Long Name", "2-alpha code", "Special Notes", "WB-2 code", "Alternative conversion factor", "PPP survey year", "Latest water withdrawal data", "National accounts base year", "National accounts reference year", "SNA price valuation", "Lending category", "Other groups", "System of National Accounts", "Balance of Payments Manual in use", "System of trade",	"Government Accounting concept", "IMF data dissemination standard",	"Latest population census", "Latest household survey", "Source of most recent Income and expenditure data",	"Vital registration complete", "Latest agricultural census", "Latest industrial data", "Latest trade data", "External debt Reporting status"], axis=1)
df_country_clean.describe()

,Country Code,Region,Income Group
count,265,265,265
unique,265,8,5
top,ABW,Europe & Central Asia,High income
freq,1,58,82


In [11]:
df_country_clean.head(2)

,Country Code,Region,Income Group
0,ABW,Latin America & Caribbean,High income
1,AFE,N/A,N/A


In [12]:
# df_country_clean.to_csv("WDI_test/WDICountry_clean.csv", index=False)

#### 1.2.3 WDISeries

In [13]:
df_indicators = df_indicators.fillna('N/A')
df_indicators_clean = df_indicators.drop(["Unit of measure", "Aggregation method", "Long definition", "Periodicity", "Series Code", "Base Period", "Short definition", "Other notes", "Related source links", "Other web links",	"Related indicators", "License Type", "Limitations and exceptions",	"Notes from original source","General comments", "Source", "Statistical concept and methodology",	"Development relevance"], axis=1)
df_indicators_clean.describe()

,Topic,Indicator Name
count,1477,1477
unique,89,1477
top,Public Sector: Policy & institutions,"Agricultural machinery, tractors"
freq,76,1


In [14]:
df_indicators_clean.head(2)

,Topic,Indicator Name
0,Environment: Agricultural production,"Agricultural machinery, tractors"
1,Environment: Agricultural production,Fertilizer consumption (% of fertilizer produc...


In [15]:
# df_indicators_clean.to_csv("WDI_test/WDISeries_clean.csv", index=False)

### 1.3 Merge Datasets
#### 1.3.1 What indicators should I analyse?

In [16]:
# Lets first see for which indicators we have maximum values available
df_indicator_name_count = df_main_clean[df_main_clean['Indicator_Value'] != "N/A"]
df_indicator_name_count = df_indicator_name_count.groupby('Indicator Name')['Indicator Name'].count().reset_index(name='Count') # Count the number of values we have for each Indicator Name
df_indicators_clean_topic = df_indicators_clean[["Topic", "Indicator Name"]]
df_indicator_name_count = df_indicator_name_count.merge(df_indicators_clean_topic, on="Indicator Name", how='left') # Add topic to filter by Topic name in the next step
df_indicator_name_count= df_indicator_name_count.loc[df_indicator_name_count.groupby('Topic')['Count'].idxmax()] # Get which Indicator name in each topic have the hightest count
df_indicator_name_count.head(2)

,Indicator Name,Count,Topic
461,"Foreign direct investment, net inflows (BoP, c...",11706,Economic Policy & Debt: Balance of payments: C...
920,"Net trade in goods and services (BoP, current ...",7320,Economic Policy & Debt: Balance of payments: C...


In [17]:
# Get the list of unique Indicator names
indicator_names_to_filter = df_indicator_name_count['Indicator Name'].to_list()
# print(len(indicator_names_to_filter))

# Let's analyse only these 89 Indicators
df_indicator_name_filtered = df_main_clean[df_main_clean["Indicator Name"].isin(indicator_names_to_filter)]
# print(df_indicator_name_filtered .shape)

In [18]:
# Handling Null Values
df_indicator_name_filtered['Indicator_Value'] = pd.to_numeric(df_indicator_name_filtered['Indicator_Value'], errors='coerce') # Convert non-numeric Indicator values to NaN
# print(df_indicator_name_filtered['Indicator_Value'].isna().sum())

# There are still many values available for us to analyse, so lets drop the rows with Null value in 'Indicator_Value' column
df_filtered = df_indicator_name_filtered.dropna(subset=['Indicator_Value'])
# df_filtered.shape

In [19]:
# Do the pivot step to do the correlation
# I am aware that the correlation calculated here is not completely correct as there are many Null values and it could create a bias. However, for the filtering step, this should be good.
df_filtered_pivot = df_filtered.pivot_table(index=['Country Name', 'Country Code', 'Year'], columns='Indicator Name', values='Indicator_Value').reset_index() # Indicators(row -> column)
# print(df_filtered_pivot.shape)
df_filtered_pivot.head(2)

Indicator Name,Country Name,Country Code,Year,Adjusted savings: education expenditure (% of GNI),"Adolescent fertility rate (births per 1,000 women ages 15-19)",Age dependency ratio (% of working-age population),"Agriculture, forestry, and fishing, value added (constant 2015 US$)","Agriculture, forestry, and fishing, value added (constant LCU)","Agriculture, forestry, and fishing, value added (current LCU)","Agriculture, forestry, and fishing, value added (current US$)",...,Taxes on goods and services (% of revenue),Terrestrial protected areas (% of total land area),"Total reserves (includes gold, current US$)",Trade (% of GDP),UHC service coverage index,"Unemployment, female (% of female labor force) (modeled ILO estimate)",Women Business and the Law Index Score (scale 1-100),Women who believe a husband is justified in beating his wife (any of five reasons) (%),Women who were first married by age 18 (% of women ages 20-24),Women's share of population ages 15+ living with HIV (%)
0,Afghanistan,AFG,1960,NaN,138.876,80.051114,NaN,NaN,NaN,NaN,...,NaN,NaN,50690800.0,11.157027,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1961,NaN,138.717,80.222340,NaN,NaN,NaN,NaN,...,NaN,NaN,42444500.0,12.550610,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Perform correlation to reduce the number of indicators and see if there are relations between them
df_filtered_pivot_clean = df_filtered_pivot.select_dtypes(include=['number'])
correlation_matrix = df_filtered_pivot_clean.corr().reset_index()
correlation_matrix.columns.name = None
correlation_matrix.head(2)

,Indicator Name,Adjusted savings: education expenditure (% of GNI),"Adolescent fertility rate (births per 1,000 women ages 15-19)",Age dependency ratio (% of working-age population),"Agriculture, forestry, and fishing, value added (constant 2015 US$)","Agriculture, forestry, and fishing, value added (constant LCU)","Agriculture, forestry, and fishing, value added (current LCU)","Agriculture, forestry, and fishing, value added (current US$)","Air transport, passengers carried",Broad money (% of GDP),...,Taxes on goods and services (% of revenue),Terrestrial protected areas (% of total land area),"Total reserves (includes gold, current US$)",Trade (% of GDP),UHC service coverage index,"Unemployment, female (% of female labor force) (modeled ILO estimate)",Women Business and the Law Index Score (scale 1-100),Women who believe a husband is justified in beating his wife (any of five reasons) (%),Women who were first married by age 18 (% of women ages 20-24),Women's share of population ages 15+ living with HIV (%)
0,Adjusted savings: education expenditure (% of ...,1.000000,-0.161774,-0.079074,-0.091644,-0.051388,-0.017944,-0.067771,0.019380,0.088298,...,-0.176924,-0.029103,-0.022233,0.180811,0.210321,0.103591,0.185113,-0.049844,-0.219945,-0.088292
1,"Adolescent fertility rate (births per 1,000 wo...",-0.161774,1.000000,0.758132,-0.098403,-0.035349,-0.039757,-0.114181,-0.168252,-0.574944,...,-0.073652,-0.005386,-0.170313,-0.297144,-0.751731,-0.129082,-0.480648,0.345025,0.862421,0.657251


In [21]:
# Find interesting correlations and get those indicators
filtered_correlated_columns = []
for c in correlation_matrix.columns:
    if c != "Indicator Name":
        for i in range(len(correlation_matrix)):
            if correlation_matrix[c][i] > 0.7 or correlation_matrix[c][i] < -0.7:
                if correlation_matrix[c][i] != 1:
                    filtered_correlated_columns.append(c)
                    # print(c, ' - ', correlation_matrix["Indicator Name"][i], ' - ', correlation_matrix[c][i])

filtered_correlated_columns = np.unique(filtered_correlated_columns)
# print(len(filtered_correlated_columns))
# print(filtered_correlated_columns)

In [22]:
# Manually filter out columns of interest (I chose to analyse 10 indicators)
indicator_names_finalized = ["Agriculture, forestry, and fishing, value added (current US$)",
                             "GDP per capita (current US$)",
                             "International tourism, number of arrivals",
                             "Merchandise exports (current US$)",
                             "Military expenditure (current USD)",
                             "Electricity production from renewable sources, excluding hydroelectric (kWh)",
                             "Primary education, pupils",
                             "Survival to age 65, female (% of cohort)",
                             "Prevalence of anemia among children (% of children ages 6-59 months)",
                             "Adolescent fertility rate (births per 1,000 women ages 15-19)"]

df_filtered_main = df_filtered[df_filtered["Indicator Name"].isin(indicator_names_finalized)]
df_filtered_main = df_filtered_main.dropna(subset=['Indicator_Value'])
# print(df_filtered_main.shape)
df_filtered_main.head(2)

,Country Name,Country Code,Indicator Name,Year,Indicator_Value
50,Africa Eastern and Southern,AFE,"Adolescent fertility rate (births per 1,000 wo...",1960,140.180526
480,Africa Eastern and Southern,AFE,GDP per capita (current US$),1960,161.638982


In [23]:
# Merge WDICSV, WDICountry and WDISeries (cleaned and filtered df)
df = df_filtered_main.merge(df_country_clean, on='Country Code', how='left')
df = df.merge(df_indicators_clean, on='Indicator Name', how='left')
df.head(2)

,Country Name,Country Code,Indicator Name,Year,Indicator_Value,Region,Income Group,Topic
0,Africa Eastern and Southern,AFE,"Adolescent fertility rate (births per 1,000 wo...",1960,140.180526,N/A,N/A,Health: Reproductive health
1,Africa Eastern and Southern,AFE,GDP per capita (current US$),1960,161.638982,N/A,N/A,Economic Policy & Debt: National accounts: US$...


#### 1.3.2 Clean Merged dataset

In [24]:
df = df[["Region", "Country Code", "Country Name", "Income Group", "Year", "Topic", "Indicator Name", "Indicator_Value"]]
df.head(2)

,Region,Country Code,Country Name,Income Group,Year,Topic,Indicator Name,Indicator_Value
0,N/A,AFE,Africa Eastern and Southern,N/A,1960,Health: Reproductive health,"Adolescent fertility rate (births per 1,000 wo...",140.180526
1,N/A,AFE,Africa Eastern and Southern,N/A,1960,Economic Policy & Debt: National accounts: US$...,GDP per capita (current US$),161.638982


In [25]:
# I see that there are Regions with N/A - So I want to see which are those
region_to_country_mapping = df.groupby('Region')['Country Name'].unique().to_dict()

for region, countries in region_to_country_mapping.items():
    if region =="N/A":
        print('Region:', region, 'Countries: ', ", ".join(countries))

Region: N/A Countries:  Africa Eastern and Southern, Africa Western and Central, Arab World, Caribbean small states, Central Europe and the Baltics, Early-demographic dividend, East Asia & Pacific, East Asia & Pacific (excluding high income), East Asia & Pacific (IDA & IBRD countries), Euro area, Europe & Central Asia, Europe & Central Asia (excluding high income), Europe & Central Asia (IDA & IBRD countries), European Union, Fragile and conflict affected situations, Heavily indebted poor countries (HIPC), High income, IBRD only, IDA & IBRD total, IDA blend, IDA only, IDA total, Late-demographic dividend, Latin America & Caribbean, Latin America & Caribbean (excluding high income), Latin America & the Caribbean (IDA & IBRD countries), Least developed countries: UN classification, Low & middle income, Low income, Lower middle income, Middle East & North Africa, Middle East & North Africa (excluding high income), Middle East & North Africa (IDA & IBRD countries), Middle income, North Ame

In [26]:
# There are a few Country Names that I don't want to analyse
country_names_to_remove = ["OECD members", "Heavily indebted poor countries (HIPC)", "High income", "IBRD only", "IDA & IBRD total", "IDA blend", "IDA only", "IDA total", "Late-demographic dividend", "Fragile and conflict affected situations", "Early-demographic dividend", "Least developed countries: UN classification", "Low & middle income", "Low income", "Lower middle income", "Middle income", "Post-demographic dividend", "Pre-demographic dividend", "Upper middle income", "World"]
df = df[~df['Country Name'].isin(country_names_to_remove)]

# Change Regions with N/A to "Others"
df['Region'] = df['Region'].replace('N/A', 'Others')

df.loc[df['Country Name'].str.contains('Africa'), 'Region'] = 'Sub-Saharan Africa'
df.loc[df['Country Name'].str.contains('Arab'), 'Region'] = 'Middle East & North Africa'
df.loc[df['Country Name'].str.contains('Caribbean'), 'Region'] = 'North America'
df.loc[df['Country Name'].str.contains('Euro'), 'Region'] = 'Europe & Central Asia'
df.loc[df['Country Name'].str.contains('Pacific'), 'Region'] = 'East Asia & Pacific'
df.loc[df['Country Name'].str.contains('Middle East & North Africa'), 'Region'] = 'Middle East & North Africa'
df.loc[df['Country Name'].str.contains('North America'), 'Region'] = 'North America'
df.loc[df['Country Name'].str.contains('South Asia'), 'Region'] = 'South Asia'

df.head(2)

,Region,Country Code,Country Name,Income Group,Year,Topic,Indicator Name,Indicator_Value
0,Sub-Saharan Africa,AFE,Africa Eastern and Southern,N/A,1960,Health: Reproductive health,"Adolescent fertility rate (births per 1,000 wo...",140.180526
1,Sub-Saharan Africa,AFE,Africa Eastern and Southern,N/A,1960,Economic Policy & Debt: National accounts: US$...,GDP per capita (current US$),161.638982


In [27]:
# df.to_csv("WDI_test/WDI_final_merged_data.csv", index=False)

In [28]:
# Have a pivoted version of df as well
df_temp = df.copy()
df_temp.drop(['Topic'], axis=1, inplace=True)
df_temp['Indicator_Value'] = pd.to_numeric(df_temp['Indicator_Value'], errors='coerce') # Convert non-numeric Indicator values to NaN
df_pivot = df_temp.pivot_table(index=['Region', 'Country Name', 'Country Code', 'Year', 'Income Group'], columns='Indicator Name', values='Indicator_Value').reset_index() # Indicators(row -> column)
df_pivot.columns.name = None
df_pivot.head(2)

,Region,Country Name,Country Code,Year,Income Group,"Adolescent fertility rate (births per 1,000 women ages 15-19)","Agriculture, forestry, and fishing, value added (current US$)","Electricity production from renewable sources, excluding hydroelectric (kWh)",GDP per capita (current US$),"International tourism, number of arrivals",Merchandise exports (current US$),Military expenditure (current USD),Prevalence of anemia among children (% of children ages 6-59 months),"Primary education, pupils","Survival to age 65, female (% of cohort)"
0,East Asia & Pacific,American Samoa,ASM,1960,High income,53.75,NaN,NaN,NaN,NaN,8000000.0,NaN,NaN,NaN,67.573781
1,East Asia & Pacific,American Samoa,ASM,1961,High income,56.67,NaN,NaN,NaN,NaN,9000000.0,NaN,NaN,NaN,70.261239


In [29]:
# Handle Indicator Values with Null value
# I see that there are columns with Null values - let me get their names
df_pivot_clean = df_pivot.copy()
df_pivot_clean['Year'] = df_pivot_clean['Year'].astype(int)
columns_with_null_indicator_values = df_pivot_clean.columns.to_list()
# print(len(columns_with_null_indicator_values))
columns_with_null_indicator_values = [c for c in columns_with_null_indicator_values if c not in ['Region', 'Country Name', 'Country Code', 'Year', 'Income Group']]
# print(len(columns_with_null_indicator_values))

# Replace Null values
df_pivot_clean.sort_values(by=['Country Name', 'Year'], inplace=True)
for c in columns_with_null_indicator_values:
    if df_pivot_clean[c].isna().sum() > 0:
        rolling_median = df_pivot_clean.groupby('Country Name')[c].apply(lambda x: x.rolling(window=5, min_periods=1).median())# Calculate the rolling median
        df_pivot_clean[c] = df_pivot_clean.apply(lambda row: rolling_median[row['Country Name']].get(row.name) if pd.isna(row[c]) and pd.notna(rolling_median[row['Country Name']].get(row.name)) else row[c], axis=1)# Replace missing values with the calculated rolling median

        df_pivot_clean[c] = df_pivot_clean.groupby('Country Name')[c].transform(lambda x: x.fillna(x.median()) if x.isnull().any() else x)# Replace missing values with the calculated median of that country

        df_pivot_clean[c] = df_pivot_clean[c].transform(lambda x: x.fillna(x.median()) if x.isnull().any() else x)# Replace missing values with the calculated median of the column

In [30]:
df_pivot_clean = df_pivot_clean.rename(columns = {"Adolescent fertility rate (births per 1,000 women ages 15-19)": "Adolescent fertility rate", 
                                                  "Agriculture, forestry, and fishing, value added (current US$)":"Primary Sector Value", 
                                                  "Electricity production from renewable sources, excluding hydroelectric (kWh)":"Non-Hydro Renewable Electricity",
                                                  "GDP per capita (current US$)": "GDP per capita",
                                                  "International tourism, number of arrivals": "Number of arrivals",
                                                  "Merchandise exports (current US$)":"Merchandise exports",
                                                  "Military expenditure (current USD)":"Military expenditure",
                                                  "Prevalence of anemia among children (% of children ages 6-59 months)":"Anemia among children",
                                                  "Primary education, pupils":"Primary education",
                                                  "Survival to age 65, female (% of cohort)":"Female 65+ Survival"})

indicator_names_finalized = ["Adolescent fertility rate",
                             "Primary Sector Value",
                             "Non-Hydro Renewable Electricity",
                             "GDP per capita",
                             "Number of arrivals",
                             "Merchandise exports",
                             "Military expenditure",
                             "Anemia among children",
                             "Primary education",
                             "Female 65+ Survival"]
df_pivot_clean.head(2)

,Region,Country Name,Country Code,Year,Income Group,Adolescent fertility rate,Primary Sector Value,Non-Hydro Renewable Electricity,GDP per capita,Number of arrivals,Merchandise exports,Military expenditure,Anemia among children,Primary education,Female 65+ Survival
11460,South Asia,Afghanistan,AFG,1960,Low income,138.876,4.126481e+09,7000000.0,62.369375,1713000.0,50000000.0,1.858783e+08,44.5,1024741.0,21.190631
11461,South Asia,Afghanistan,AFG,1961,Low income,138.717,4.126481e+09,7000000.0,62.443703,1713000.0,53000000.0,1.858783e+08,44.5,1024741.0,21.819029


## 2 Data Visualization
### 2.1 Heatmap

In [48]:
# df_heatmap = df_pivot_clean.copy().select_dtypes(include=['number'])
# df_heatmap.drop(["Year"], axis=1, inplace=True)
# df_heatmap_log = df_heatmap.applymap(lambda x: 0 if pd.isna(x) else np.log(x + 1))
# df_heatmap_log = (df_heatmap_log - df_heatmap_log.min()) / (df_heatmap_log.max() - df_heatmap_log.min())
# df_heatmap_log.head(2)

In [49]:
# plt.figure(figsize=(8, 6)) 

# corr_matrix = df_heatmap_log.corr()
# mask = np.zeros_like(corr_matrix)
# mask[np.triu_indices_from(mask)] = True
# mask[np.diag_indices_from(mask)] = False

# sns.heatmap(corr_matrix, cmap="coolwarm", annot=True, vmin=-1, vmax=1, annot_kws={"size":10}, mask=mask)
# plt.xticks(fontsize=10, ha="right", rotation=50)
# plt.yticks(fontsize=10)
# # plt.savefig("Visuals/heatmap.png")
# plt.suptitle("Correlation Heatmap              ", fontsize=16)
# plt.title("Relationship between Indicators\n", fontsize=12, y=0.98)
# plt.show()

### 2.2 Pairplot

In [42]:
# plt.figure(figsize=(8, 6))
# sns.set(font_scale=1.5)
# sns.pairplot(df_heatmap_log, diag_kind="kde")
# plt.show()

In [44]:
# variable_pairs = [(var1, var2) for var1 in df_heatmap_log.columns for var2 in df_heatmap_log.columns if var1 != var2]

# plt.figure(figsize=(5, 3)) 
# for i, (var1, var2) in enumerate(variable_pairs):
#     sns.regplot(x=var1, y=var2, data=df_heatmap_log, scatter_kws={"alpha": 0.3}, line_kws={"linewidth": 2, "color": "red"})
#     plt.title(f"{var1} vs {var2}", fontsize=14)
#     plt.show()

### 2.3 Distplot

In [50]:
# df_distplot = df_pivot_clean.copy()
# df_distplot[indicator_names_finalized] = df_heatmap_log[indicator_names_finalized]

# num_cols_per_row = 4

# for indicator in indicator_names_finalized:
#     # plt.figure(figsize=(3, 5)) 
#     num_rows = (len(df_distplot['Region'].unique()) + num_cols_per_row - 1) // num_cols_per_row
#     g = sns.FacetGrid(df_distplot, col='Region', sharex=False, col_wrap=num_cols_per_row, height=4)
#     print(f"{indicator}: ")
#     for region_name, ax in zip(df_distplot['Region'].unique(), g.axes):
#         sns.distplot(df_distplot[df_distplot['Region'] == region_name][indicator], ax=ax) 
#         ax.set_title(region_name, fontsize=16) 
#         ax.set_xlim(0, 1)
#         ax.set_xlabel('')
#         ax.set_ylabel('Density', fontsize=14)
        
#         print(f"{region_name}: ")
#         print("Skew:", df_distplot[df_distplot['Region'] == region_name][indicator].skew())
#         print("Kurtosis:", df_distplot[df_distplot['Region'] == region_name][indicator].kurt())

#     plt.tight_layout()
#     plt.suptitle(f"Distribution of {indicator} for each region", y=1.05, fontsize=18)
#     plt.show()

In [ ]:
# for indicator in indicator_names_finalized:
#     for region in df_distplot['Region'].unique():
#         temp_distplot = df_distplot[df_distplot['Region'] == region]
#         sns.distplot(temp_distplot[indicator], label=region)
#         plt.xlabel('')
#     print(indicator, ": ")
#     plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
#     plt.show()

### 2.4 Box plot

In [52]:
# df_boxplot = df_pivot_clean.copy()
# df_boxplot[indicator_names_finalized] = df_heatmap_log[indicator_names_finalized]

# num_cols_per_row = 4

# for indicator in indicator_names_finalized:
#     # plt.figure(figsize=(3, 5)) 
#     num_rows = (len(df_boxplot['Region'].unique()) + num_cols_per_row - 1) // num_cols_per_row
#     g = sns.FacetGrid(df_boxplot, col='Region', sharex=False, col_wrap=num_cols_per_row, height=4)
#     for region_name, ax in zip(df_boxplot['Region'].unique(), g.axes):
#         sns.boxplot(df_boxplot[df_boxplot['Region'] == region_name][indicator], ax=ax) 
#         ax.set_title(region_name, fontsize=16) 
#         ax.set_ylabel(indicator, fontsize=14)

#     plt.tight_layout()
#     plt.suptitle(f"Distribution of {indicator} for each region", y=1.05, fontsize=18)
#     plt.show()

### 2.5 Bar plots

In [53]:
# df_barplot = df_pivot_clean.copy()
# df_barplot[indicator_names_finalized] = df_heatmap_log[indicator_names_finalized]

# for indicator in indicator_names_finalized:
#     for region in df_barplot['Region'].unique():
#         df_barplot_region = df_boxplot[df_boxplot['Region'] == region]
#         region_countries = df_barplot_region['Country Name'].unique()
#         z_scores = df_barplot_region.groupby('Country Name')[indicator].apply(lambda x: (x - x.mean()) / x.std())
#         outliers = (z_scores.abs() > 3) 

#         outliers_count = outliers.groupby('Country Name').sum().astype(int).reset_index()
#         outliers_count_filtered = outliers_count[outliers_count[indicator] > 0]
#         if len(outliers_count_filtered)>3:
#             outliers_count_filtered.sort_values(by=indicator, ascending=False, inplace=True)

#             plt.figure(figsize=(8, 5))
#             sns.barplot(x='Country Name', y=indicator, data=outliers_count_filtered, palette = "rocket")
#             plt.title(f'Number of Outliers for {indicator} in {region}')
#             plt.xlabel('Country Name')
#             plt.xticks(fontsize=10, ha="right", rotation=65)
#             plt.ylabel('Number of Outliers')
#             plt.show()

### 2.6 Line chart

In [ ]:
# df_linechart = df_pivot_clean.copy()
# df_linechart['Year'] = df_linechart['Year'].astype(int)
# plt.figure(figsize=(8, 6))

# for indicator in indicator_names_finalized:
#     temp = df_linechart[['Year', 'Region', indicator]]
#     sns.lineplot(x='Year', y=indicator, hue='Region', data=temp)
#     plt.title(indicator)
#     plt.xlabel('Year', fontsize=8)
#     plt.ylabel(indicator, fontsize=8)
#     plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
#     plt.savefig("Visuals/linechart_{indicator}.png")
#     plt.show()

### 2.7 Parallel Coordinate plot

In [54]:
# df_parallel_coordinate = df_pivot_clean.copy()
# df_parallel_coordinate[indicator_names_finalized] = df_heatmap_log[indicator_names_finalized]
# region_mapping = {'South Asia': 2, 'Sub-Saharan Africa': 1, 'Europe & Central Asia': 4, 'Middle East & North Africa': 5, 'East Asia & Pacific': 6, 'Latin America & Caribbean': 7, 'North America': 3, 'Others': 8}
# df_parallel_coordinate['Region_encoded'] = [region_mapping[i] for i in df_parallel_coordinate['Region']]
# print("Region Mapping: ", region_mapping)

In [ ]:
# df_parallel_coordinate = df_pivot_clean.copy()
# df_parallel_coordinate[indicator_names_finalized] = df_heatmap_log[indicator_names_finalized]
# region_mapping = {'South Asia': 2, 'Sub-Saharan Africa': 1, 'Europe & Central Asia': 4, 'Middle East & North Africa': 5, 'East Asia & Pacific': 6, 'Latin America & Caribbean': 7, 'North America': 3, 'Others': 8}
# df_parallel_coordinate['Region_encoded'] = [region_mapping[i] for i in df_parallel_coordinate['Region']]
# print("Region Mapping: ", region_mapping)

# df_parallel_coordinate  = df_parallel_coordinate .select_dtypes(include=['number'])
# plt.figure(figsize=(20, 6))
# fig = px.parallel_coordinates(df_parallel_coordinate, 
#                               dimensions=["Year", "Primary Sector Value", "GDP per capita", "Number of arrivals", "Merchandise exports", "Military expenditure", "Non-Hydro Renewable Electricity", "Primary education", "Female 65+ Survival", "Anemia among children", "Adolescent fertility rate", "Region_encoded"],
#                               labels={"Year": 'Y', "Primary Sector Value":'P', "GDP per capita":'G', "Number of arrivals":'A', "Merchandise exports":'M', "Military expenditure":'E', "Non-Hydro Renewable Electricity":'H', "Primary education":'D', "Female 65+ Survival":'S', "Anemia among children":'C', "Adolescent fertility rate":'F', "Region_encoded":'R'},
#                               color="Region_encoded", 
#                               title="Parallel Coordinate chart: WDI Indicators",
#                               color_continuous_scale=px.colors.sequential.RdBu,
#                               color_continuous_midpoint=4)
# fig.update_layout(font=dict(size=13))
# fig.show()

### 2.8 Perform PCA

In [55]:
# df_temp = df_pivot_clean.copy()
# region_mapping = {'South Asia': 2, 'Sub-Saharan Africa': 1, 'Europe & Central Asia': 4, 'Middle East & North Africa': 5, 'East Asia & Pacific': 6, 'Latin America & Caribbean': 7, 'North America': 3, 'Others': 8}
# df_temp['Region_encoded'] = [region_mapping[i] for i in df_temp['Region']]

# # Scale - remove mean and scale to unit variance
# df_pivot_numeric = df_temp.select_dtypes(include=['number'])
# values = StandardScaler().fit_transform(df_pivot_numeric)

# # indicator_mapping = {"Adolescent fertility rate":"A", "Primary Sector Value":"B",	"Non-Hydro Renewable Electricity":"C", "GDP per capita":"D", "Number of arrivals":"E", "Merchandise exports":"F", "Military expenditure":"G", "Anemia among children":"H", "Primary education":"I", "Female 65+ Survival":"J"}
# # df_pivot_numeric = df_pivot_numeric.rename(indicator_mapping, axis=1)

# df_pca = pd.DataFrame(values, columns=df_pivot_numeric.columns)
# df_pca.drop(['Year', 'Region_encoded'], axis=1, inplace=True) 

In [56]:
# # Capture 95% of the explained variance
# model = pca(n_components=0.95)
# components = model.fit_transform(df_pca)

# # Get the explained variance
# total_variance = np.var(df_pca, axis=0).sum()
# explained_variances = np.var(components['PC'], axis=0) / total_variance

# # Create scatter plot
# fig, ax = model.biplot(n_feat=10,
#                        labels = df_pivot['Region'],
#                        legend=True)

In [ ]:
# pca = PCA(n_components=0.95)
# components = pca.fit_transform(values)

# # Get the explained variance
# total_variance = np.var(df_pca, axis=0).sum()
# explained_variances = np.var(components, axis=0) / total_variance

# # Create scatter plot with bold features
# fig, ax = plt.subplots(figsize=(10,5))
# for i, feature in enumerate(df_pca.columns):
#     ax.arrow(0, 0, pca.components_[0, i], pca.components_[1, i], color='r', alpha=0.5, linewidth=0.5, width=0.005)
#     ax.text(pca.components_[0, i]*1.2, pca.components_[1, i]*1.2, f'{feature}', color='k', ha='center', va='center', fontsize=10)

# # Add labels and title
# ax.set_xlabel('Principal Component 1', fontsize=10)
# ax.set_xlim([-0.5,0.6])
# ax.set_ylim([-0.5,0.6])
# ax.tick_params(labelsize=10)
# ax.set_ylabel('Principal Component 2', fontsize=10)
# ax.set_title('PCA Biplot', fontsize=16)

# plt.show()


In [57]:
# # Plot explained variance
# plt.figure(figsize=(2,2))
# fig, ax = model.plot()
# plt.grid(False)
# plt.show()

In [58]:
# # Just to see in the 3D space if there is a clear separation
# pca = PCA(n_components=10)
# components = pca.fit_transform(df_pca)
# total_var = pca.explained_variance_ratio_.sum() * 100
# fig = px.scatter_3d(components, x=0, y=1, z=2, 
#                     color=df_pivot_numeric['Region_encoded'],
#                     title=f'Total Explained Variance: {total_var:.2f}%',
#                     labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'})
# fig.show()

In [59]:
# # perform weighted average and get the combined PCA in an array
# PC = []
# for i in range(8):
#     PC.append(components[:,i]*explained_variances[i])
# df_pivot_clean['PC'] = sum(PC)
# df_pivot_clean.head(2)

### 2.9 Perform t-SNE

In [ ]:
# df_pivot_numeric = df_parallel_coordinate.select_dtypes(include=['number'])
# df_pivot_numeric.drop(['Year', 'Region_encoded'], axis=1, inplace=True)  # Drop non-numeric columns

# values = StandardScaler().fit_transform(df_pivot_numeric)


# tsne = TSNE(n_components=2, perplexity=50, learning_rate=200, random_state=42)
# tsne_result = tsne.fit_transform(values)


# df_tsne = pd.DataFrame(tsne_result, columns=['t-SNE1', 't-SNE2'])


# df_combined_tsne = pd.concat([df_parallel_coordinate[['Year', 'Region_encoded']], df_tsne], axis=1)

# # Create scatter plot
# plt.figure(figsize=(10, 8))
# scatter = plt.scatter(df_combined_tsne['t-SNE1'], df_combined_tsne['t-SNE2'], c=df_combined_tsne['Region_encoded'], cmap='viridis')
# plt.colorbar(scatter, label='Region_encoded')
# plt.title('t-SNE Scatter Plot')
# plt.xlabel('t-SNE1')
# plt.ylabel('t-SNE2')
# plt.show()


### 2.10 Chorolpeth chart

In [60]:
# # The combined effect of these indicators can in a way show a country's/region's economic development
# # I am interested to see if there are any differences in the rate of economic development when compared to other countries between 1960-1990 and 1990-2022
# world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')) # Load a world shapefile using geopandas
# df_chorolpeth = df_pivot_clean.copy()

In [61]:
# df_chorolpeth['PC'] = df_chorolpeth['PC'].apply(lambda x: 0 if pd.isna(x) else np.log(x + 1))
# min_val = df_chorolpeth['PC'].min()
# max_val = df_chorolpeth['PC'].max()
# df_chorolpeth['PC'] = 2 * (df_chorolpeth['PC'] - min_val) / (max_val - min_val) - 1

#### 2.8.1 1960 - 1990

In [62]:
# df_chorolpeth_1960 = df_chorolpeth.copy()
# df_chorolpeth_1960 = df_chorolpeth_1960[df_chorolpeth_1960['Year']<=1990]
# merged_df_chorolpeth_1960 = world.set_index('iso_a3').join(df_chorolpeth_1960.set_index('Country Code')) # Merge the geopandas data with WDI data

In [63]:
# fig, ax = plt.subplots(1, 1, figsize=(15, 5))
# merged_df_chorolpeth_1960.plot(column='PC', cmap='RdYlBu', ax=ax, legend=True, edgecolor='0.8',  linewidth=0.1, vmin=-1, vmax=1)
# ax.set_title("Economic Development Rate across Countries: 1960 - 1990", fontsize=14)
# ax.tick_params(axis='both', labelsize=10)
# plt.show()

#### 2.8.1 1991 - 2022

In [64]:
# df_chorolpeth_1991 = df_chorolpeth.copy()
# df_chorolpeth_1991 = df_chorolpeth_1991[df_chorolpeth_1991['Year']>1990]
# merged_df_chorolpeth_1991 = world.set_index('iso_a3').join(df_chorolpeth_1991.set_index('Country Code')) # Merge the geopandas data with WDI data

In [65]:
# fig, ax = plt.subplots(1, 1, figsize=(15, 5))
# merged_df_chorolpeth_1991.plot(column='PC', cmap='RdYlBu', ax=ax, legend=True, edgecolor='0.8',  linewidth=0.1, vmin=-1, vmax=1)
# ax.set_title("Economic Development Rate across Countries: 1991 - 2022", fontsize=14)
# ax.tick_params(axis='both', labelsize=10)
# plt.show()

### 3 Validation

Is it related to the income group differences?

In [ ]:
# # Check: Does the economic development have anything to do with income group differences?
# # Lets see the correlation 
# df_validation = df_pivot_clean.copy()
# df_validation = df_validation[df_validation['Income Group'] != 'N/A']

In [ ]:
# income_group_mapping = {'Low income':1, 'Lower middle income': 2, 'Upper middle income':3, 'High income':4}
# df_validation['Income_group_encoded'] = [income_group_mapping[i] for i in df_validation['Income Group']]
# print("Income group Mapping: ", income_group_mapping)

In [ ]:
# plt.figure(figsize=(8, 6))  
# df_validation_numeric = df_validation.select_dtypes(include=['number'])
# df_validation_numeric.drop(['Year'], axis=1, inplace=True)
# sns.heatmap(df_validation_numeric.corr(), annot=True, vmin=-1, vmax=1, annot_kws={"size": 10})
# plt.xticks(fontsize=10)
# plt.yticks(fontsize=10)
# plt.show()

In [67]:
# Conclusion: the economic development is not correlated much with whether the country falls in high income region or low income region